## Paul Andrew Plaground

[paul@mrpaulandrew.com](mailto:paul@mrpaulandrew.com)

Playing around with the Spark.Net basics, using the examples from the Spark.Net tutorial here: 
[https://dotnet.microsoft.com/](https://dotnet.microsoft.com/learn/data/spark-tutorial/intro)


In [ ]:
using Microsoft.Spark.Sql;

string inputPath = "abfss://paulplayground@mvpsynapsestorage.dfs.core.windows.net/mySparkApp/input.txt";

DataFrame rawData = spark.Read().Text(inputPath);

In [ ]:
rawData.Show()

In [ ]:
rawData.PrintSchema();

In [ ]:
DataFrame words = rawData
    .Select(Functions.Split(Functions.Col("value"), " ").Alias("words"))
    .Select(Functions.Explode(Functions.Col("words"))
    .Alias("word"))
    .GroupBy("word")
    .Count()
    .OrderBy(Functions.Col("count").Desc());

words.Show();

In [ ]:
words.CreateOrReplaceTempView("wordView");

DataFrame sqlDf = spark.Sql("SELECT * FROM wordView WHERE count > 1");
sqlDf.Show();

In [ ]:
string outputPath = "abfss://paulplayground@mvpsynapsestorage.dfs.core.windows.net/mySparkApp/wordsSummary.parquet";

words.Write().Mode(SaveMode.Overwrite).Parquet(outputPath);

In [ ]:
spark.Stop();